In [1]:
!pip install streamlit pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.3 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=dd1cc3b19eabc2068a66a63dace5c60581dc69bea45918923abb44d243643427
  Stored in directory: /root/.cache/pip/wheels/0d/1f/e9/3ce954f5c5d9d30ec279ad8bc5d44666a64fb4be782cb39a2d
Successfully built pyngrok


In [2]:
%%writefile chatbot.py
import json
import random
import re
from difflib import SequenceMatcher
from typing import List, Dict, Tuple

class KeywordChatbot:
    def __init__(self, intents: List[Dict], fuzzy: bool = True, threshold: float = 0.85):
        self.intents = intents
        self.fuzzy = fuzzy
        self.threshold = threshold

    def _preprocess(self, text: str) -> Tuple[str, set]:
        text = text.lower().strip()
        text_no_punct = re.sub(r"[^\w\s]", " ", text)
        tokens = set(text_no_punct.split())
        return text, tokens

    def _keyword_hit(self, kw: str, text: str, tokens: set) -> int:
        kw = kw.lower().strip()
        if " " in kw:  # phrase match
            return 1 if kw in text else 0
        if kw in tokens:  # exact match
            return 1
        if self.fuzzy:  # fuzzy match for typos
            for t in tokens:
                if SequenceMatcher(a=kw, b=t).ratio() >= self.threshold:
                    return 1
        return 0

    def _best_intent(self, user_text: str) -> Tuple[str, int]:
        text, tokens = self._preprocess(user_text)
        best_tag, best_score = None, 0

        for intent in self.intents:
            score = sum(self._keyword_hit(kw, text, tokens) for kw in intent.get("keywords", []))
            if score > best_score:
                best_score, best_tag = score, intent.get("tag")

        return best_tag, best_score

    def reply(self, user_text: str) -> str:
        tag, score = self._best_intent(user_text)
        if not tag or score == 0:
            return "Sorry, I didn't catch that. Try 'hello', 'thanks', or 'bye'."
        for intent in self.intents:
            if intent.get("tag") == tag:
                return random.choice(intent.get("responses", ["..."]))
        return "..."

def load_intents(path: str = "intents.json") -> List[Dict]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data.get("intents", [])

Writing chatbot.py


In [3]:
%%writefile intents.json
{
  "intents": [
    {
      "tag": "greet",
      "keywords": ["hello", "hi", "hey", "salam", "assalamualaikum", "good morning"],
      "responses": ["Hello! 👋", "Hi there!", "Hey! How can I help?"]
    },
    {
      "tag": "goodbye",
      "keywords": ["bye", "goodbye", "see you", "later"],
      "responses": ["Bye! 👋", "See you later!", "Take care!"]
    },
    {
      "tag": "thanks",
      "keywords": ["thanks", "thank you", "appreciate"],
      "responses": ["You're welcome!", "Anytime!", "Happy to help!"]
    },
    {
      "tag": "name",
      "keywords": ["your name", "who are you"],
      "responses": ["I'm a simple keyword bot.", "Just your friendly chatbot 🙂"]
    },
    {
      "tag": "help",
      "keywords": ["help", "what can you do"],
      "responses": ["Try greeting me, asking my name, or say bye.", "I respond to keywords like 'hello', 'thanks', 'bye'."]
    },
    {
      "tag": "joke",
      "keywords": ["joke", "funny"],
      "responses": ["Why did the function cross the road? To get to the other side effect."]
    }
  ]
}


Writing intents.json


In [4]:
%%writefile app.py
import streamlit as st
from chatbot import KeywordChatbot, load_intents

st.set_page_config(page_title="Keyword Chatbot", page_icon="🤖")
st.title("Keyword Chatbot 🤖")
st.caption("Simple keyword-based chatbot running in Colab!")


intents = load_intents("intents.json")
bot = KeywordChatbot(intents=intents, fuzzy=True, threshold=0.85)

if "chat" not in st.session_state:
    st.session_state.chat = []


user_input = st.text_input("You:", placeholder="Say hello, ask for a joke, or type 'bye'...")
if st.button("Send") and user_input.strip():
    reply = bot.reply(user_input)
    st.session_state.chat.append(("You", user_input))
    st.session_state.chat.append(("Bot", reply))


for speaker, msg in st.session_state.chat:
    st.markdown(f"**{speaker}:** {msg}")


Writing app.py


# Task
Explain the error in the selected code, resolve the issue, and explain the corrected code.

## Verify ngrok authtoken

### Subtask:
Double-check that the NGROK_AUTHTOKEN is correctly added to Colab secrets and that there are no typos.


## Check streamlit process

### Subtask:
Confirm that the Streamlit app is running in the background after executing the cell.


**Reasoning**:
Execute a shell command to list processes and filter for those containing "streamlit" to confirm if the Streamlit app is running in the background.



In [9]:
!ps aux | grep streamlit

root        3438  0.4  0.5  84368 68404 ?        S    10:48   0:00 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501
root        4133  0.0  0.0   7376  3484 ?        S    10:50   0:00 /bin/bash -c ps aux | grep streamlit
root        4135  0.0  0.0   6484  2372 ?        S    10:50   0:00 grep streamlit


## Check ngrok connection

### Subtask:
Verify if ngrok is able to establish a connection to the specified port.


**Reasoning**:
Attempt to establish an ngrok connection to port 8501 and print the public URL or any error encountered.



In [10]:
try:
    public_url = ngrok.connect(8501)
    print("👉 Open your chatbot here:", public_url.public_url)
except Exception as e:
    print("Error connecting with ngrok:", e)


ERROR:pyngrok.process:t=2025-09-02T10:51:08+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"



Error connecting with ngrok: The ngrok process errored on start: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.


## Provide error message

### Subtask:
Provide the exact error message encountered when trying to set the ngrok authtoken.


## Summary:

### Data Analysis Key Findings

*   Manual verification of the `NGROK_AUTHTOKEN` in Colab secrets was required and could not be performed programmatically.
*   The Streamlit app was confirmed to be running in the background on port 8501, as indicated by the presence of the process `/usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501`.
*   Attempting to connect ngrok to port 8501 failed with the error `ERR_NGROK_4018`, indicating an authentication failure: "Usage of ngrok requires a verified account and authtoken."
*   An `AttributeError: module 'ngrok' has no attribute 'set_auth_token'` was encountered when attempting to set the ngrok authtoken, suggesting that the imported `ngrok` module does not have this function.

### Insights or Next Steps

*   Manually verify that the `NGROK_AUTHTOKEN` is correctly added and accessible within the Colab environment's secrets.
*   Investigate the correct method or function to set the authentication token for the specific `ngrok` library version being used, as `set_auth_token` appears to be incorrect.


In [11]:
!ps aux | grep streamlit

root        3438  0.2  0.5  84368 68404 ?        S    10:48   0:00 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501
root        4537  0.0  0.0   7376  3536 ?        S    10:52   0:00 /bin/bash -c ps aux | grep streamlit
root        4539  0.0  0.0   6484  2304 ?        S    10:52   0:00 grep streamlit


In [13]:
from pyngrok import ngrok
from google.colab import userdata

try:
    # Set the ngrok authtoken
    ngrok.set_auth_token(userdata.get("NGROK_AUTHTOKEN"))
    public_url = ngrok.connect(8501)
    print("👉 Open your chatbot here:", public_url.public_url)
except Exception as e:
    print("Error connecting with ngrok:", e)

ERROR:pyngrok.process:t=2025-09-02T10:53:13+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your ngrok-agent version \"2.3.41\" is too old. The minimum supported agent version for your account is \"3.7.0\". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.\r\n\r\nERR_NGROK_121\r\n"



Error connecting with ngrok: The ngrok process errored on start: Your ngrok-agent version "2.3.41" is too old. The minimum supported agent version for your account is "3.7.0". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.\r\n\r\nERR_NGROK_121\r\n.


In [1]:
!pip install --upgrade pyngrok

In [2]:
import os
token = os.environ.get("NGROK_AUTHTOKEN")
!ngrok config add-authtoken $token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from pyngrok import ngrok

!pkill streamlit || echo "no streamlit running"
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

public_url = ngrok.connect(8501)
print("👉 Open your chatbot here:", public_url.public_url)


no streamlit running


ERROR:pyngrok.process.ngrok:t=2025-09-02T10:56:59+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: None\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-02T10:56:59+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: None\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-02T10:56:59+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: None\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [4]:
import os
from google.colab import userdata

# Get the authtoken from Colab secrets
token = userdata.get("NGROK_AUTHTOKEN")

# Set the authtoken using the ngrok command line
# This writes the token to the ngrok configuration file
!ngrok config add-authtoken $token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
import os
from google.colab import userdata

# Get the authtoken from Colab secrets
token = userdata.get("NGROK_AUTHTOKEN")

# Set the authtoken using the ngrok command line
# This writes the token to the ngrok configuration file
!ngrok config add-authtoken $token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from pyngrok import ngrok
from google.colab import userdata

# Kill any existing Streamlit processes
!pkill streamlit || echo "no streamlit running"

# Run Streamlit in background
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

# Set the ngrok authtoken using pyngrok
ngrok.set_auth_token(userdata.get("NGROK_AUTHTOKEN"))

# Expose with ngrok
public_url = ngrok.connect(8501)
print("👉 Open your chatbot here:", public_url.public_url)

👉 Open your chatbot here: https://2288ea4c28f5.ngrok-free.app
